In [3]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [4]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = None

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 1200
train_size = 0.80
lookback_window = 50

In [5]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [6]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [7]:
#SETTINGS M1

hidden_parameters = 50
linear_parameters = 40
num_layers = 1

In [8]:
model = LSTM(total_params, hidden_parameters, linear_parameters, total_params, num_layers=num_layers, activation=nn.Tanh())

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor)
# print(out)

In [9]:
# SETTINGS T1
pos_weight = torch.tensor([1])
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss_function = nn.MSELoss()
n_epochs = 100

In [14]:
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1)

100%|██████████| 1150/1150 [00:30<00:00, 37.72it/s]


Epoch 1 completed. 
Training Loss: 0.147
Test Loss: 0.107


In [15]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 1150/1150 [00:12<00:00, 92.20it/s]


Evaluation results for training dataset:
8190 evaluations in total.
4448/4640 cells with earthquake activity were correctly identified. (95.86 %)
1628/3550 cells with no earthquake activity were correctly identified. (45.86 %)
1922/6370 positives turned out to be false.
192/1820 negatives turned out to be false.
Preicision: 0.698
Recall: 0.959
F1 Score: 0.808
Overall accuracy: 74.19 %
Evaluation results for training dataset:
2160 evaluations in total.
1534/1665 cells with earthquake activity were correctly identified. (92.13 %)
349/495 cells with no earthquake activity were correctly identified. (70.51 %)
146/1680 positives turned out to be false.
131/480 negatives turned out to be false.
Preicision: 0.913
Recall: 0.921
F1 Score: 0.917
Overall accuracy: 87.18 %
